In [ ]:
import bayes3d as b
import os
import jax.numpy as jnp
import jax
import bayes3d.genjax
import genjax
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
b.setup_visualizer()

In [ ]:
model_dir = os.path.join(b.utils.get_assets_dir(),"bop/ycbv/models")
obj_idx = 4
mesh_filename = os.path.join(model_dir,"obj_" + "{}".format(obj_idx+1).rjust(6, '0') + ".ply")
SCALING_FACTOR = 1.0/1000.0

In [ ]:
intrinsics = b.Intrinsics(
    height=100,
    width=100,
    fx=500.0, fy=500.0,
    cx=50.0, cy=50.0,
    near=0.01, far=50.0
)
b.setup_renderer(intrinsics)
b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)

In [ ]:
object_poses = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
            jnp.array([0.0, 0.6, 0.6]),
            jnp.array([0.0, 0.0, 0.0]),
            jnp.array([0.0, 0.0, 1.0]),
        )) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle)  for angle in jnp.linspace(-jnp.pi, jnp.pi, 7)[:-1]])
observations = b.RENDERER.render_many(object_poses[:,None,...], jnp.array([0]))

In [ ]:
b.hstack_images([b.get_depth_image(o[...,2]) for o in observations])

In [ ]:
grid = b.utils.make_translation_grid_enumeration_3d(
    -0.1, -0.1, -0.2,
    0.1, 0.1, 0.2,
    # 100, 100, 100
    60, 60, 60
)
b.show_cloud("grid", grid)

In [ ]:
voxel_occupied_occluded_free_parallel = jax.jit(jax.vmap(b.utils.voxel_occupied_occluded_free, in_axes=(0, 0, None, None, None)))

In [ ]:
occupancies = voxel_occupied_occluded_free_parallel(
    b.inverse_pose(object_poses), observations[...,2], grid, intrinsics, 0.001
)
print(occupancies.sum())

In [ ]:
b.clear()
b.show_cloud("grid", grid[(occupancies > 0.6).sum(0) > 0 ])
# b.show_cloud("grid2", grid[occupancy == 0.5],color=b.RED)

In [ ]:
mesh = b.utils.make_voxel_mesh_from_point_cloud(grid[(occupancies > 0.6).sum(0) > 0 ], 0.01 )

In [ ]:
b.clear()
b.show_trimesh("mesh", mesh)

In [ ]:
from bayes3d.viz.open3dviz import Open3DVisualizer

In [ ]:
scaled_up_intrinsics = b.camera.scale_camera_parameters(intrinsics, 2)

In [ ]:
viz = Open3DVisualizer(scaled_up_intrinsics)

In [ ]:
distinct_colors = b.viz.distinct_colors(10)

In [ ]:
viz.make_trimesh(mesh, jnp.eye(4),  jnp.array([*distinct_colors[0], 1.0]))

In [ ]:
view_poses = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
            jnp.array([0.0, 0.9, 0.9]),
            jnp.array([0.0, 0.0, 0.0]),
            jnp.array([0.0, 0.0, 1.0]),
        )) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle)  for angle in jnp.linspace(-jnp.pi, jnp.pi, 100)[:-1]])

In [ ]:
images = []
for p in tqdm(view_poses):
    images.append(viz.capture_image(intrinsics, b.inverse_pose(p)))

In [ ]:
b.viz.make_gif_from_pil_images([b.get_rgb_image(rgbd.rgb) for rgbd in images], "out.gif")

In [ ]:
b.get_rgb_image(viz.capture_image(intrinsics, b.inverse_pose(view_poses[0])).rgb)